# Berman Standard State Code Generator 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()

Required ENKI packages

In [ ]:
from thermoengine import coder

# Types of terms in a standard state properties description
There are three classes of terms:
1. Terms that apply over the whole of $T$-, $P$-space, $T_r \le T$, $P_r \le P$
2. Terms that apply over a specified range of $T$-, $P$-space, $(T_{r_\lambda},P_{r_\lambda}) \le (T,P) \le (T_\lambda,P_\lambda)$
3. Terms that apply to a specific $T_t$ and $P_t$ and higher $T$, $P$, $T_t \le T$, $P_t \le P$

Second-order phase transitions ($lambda$-transitions) are an example of the second type, as are order disorder transformations. First-order phase transitions are an example of the third type.  

## Create a model class for the Gibbs free energy

In [ ]:
model = coder.StdStateModel()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

### Define model expressions applicable over all of T,P space
An expression for the Gibbs free energy, $G(T,P)$ or the Helmholtz energy $A(T,V)$ is constructed.  The expression may have multiple parts.  Often the heat capacity function is postulated, then integrated to yield expressions for the entahlpy, entropy, and in combination the energy potential. Then, an equation of state (EOS) is adopted and that term is integrated in pressure or volume and added to the heat capacity integrals. This proceedure is follwed here.
#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: $C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3 $,
and in addition the reference condition third law entropy, $ S_{Tr,Pr} $, and enthalpy of formation from the
elements, $ \Delta H_{Tr,Pr} $, constitute additional parameters:

In [ ]:
k0,k1,k2,k3 = sym.symbols('k0 k1 k2 k3')
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
STrPr,HTrPr = sym.symbols('S_TrPr H_TrPr')

In [ ]:
CpPr

Specify paramters ...

In [ ]:
params = [('H_TrPr','J',HTrPr), ('S_TrPr','J/K',STrPr), ('k0','J/K-m',k0), ('k1','J-K^(1/2)-m',k1),
          ('k2','J-K/m',k2),  ('k3','J-K^2',k3)]

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
GPr = HTrPr + sym.integrate(CpPr,(T,Tr,T)) - T*(STrPr + sym.integrate(CpPr/T,(T,Tr,T)))

In [ ]:
GPr

... and add this expression to the model

In [ ]:
model.add_expression_to_model(GPr, params)

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space

In [ ]:
VTrPr,v1,v2,v3,v4 = sym.symbols('V_TrPr v1 v2 v3 v4')
params = [('V_TrPr', 'J/bar-m', VTrPr), ('v1','1/bar',v1), ('v2','1/bar^2',v2), ('v3','1/K',v3),  ('v4','1/K^2',v4)]

In [ ]:
GPrToP = sym.integrate(VTrPr*(1+v1*(P-Pr)+v2*(P-Pr)**2+v3*(T-Tr)+v4*(T-Tr)**2),(P,Pr,P))
model.add_expression_to_model(GPrToP, params)

### Define additional lambda heat capacity terms applicable over a restricted range of T,P space
These contributions to the potential function apply over a limited range of $T,P$ or $T,V$ space.  However, the affects of these functions propagate beyond the upper limits of the range.  Say, $f(T,P)$ is the contribution to the Gibbs free energy that describes a $\lambda$-like transition over the range $({T_{\lambda ,ref}},{P_{\lambda ,ref}})$ to $({T_\lambda },{P_\lambda })$.  Then, *above the upper limit* of temperature there is a fixed entropy contribution:  
  
$- {\left. {\frac{{\partial f\left( {T,P} \right)}}{{\partial T}}} \right|_{{T_\lambda },{P_\lambda }}} = {f_S}({T_\lambda },{P_\lambda })$  
  
and *above the upper limit of pressure* there is a fixed volume contribution:  
  
${\left. {\frac{{\partial f\left( {T,P} \right)}}{{\partial P}}} \right|_{{T_\lambda },{P_\lambda }}} = {f_V}({T_\lambda },{P_\lambda })$  

the consequence of which is that for $T > T_{\lambda}$ and $P > P_{\lambda}$, there is a contribution to the Gibbs free energy of the form:  

$- \left( {T - {T_\lambda }} \right){f_S}({T_\lambda },{P_\lambda }) + \left( {P - {P_\lambda }} \right){f_V}({T_\lambda },{P_\lambda })$  

This contribution is linear in $T$ and $P$.  

The additional energetic contributions applicable above the upper range limit will be added automatically to the model function, and need not be explicitly accounted for in building the model expressions.  

#### (1) $\lambda$-transition-like heat capacity integrals 
Parameters of the Berman (1988) lambda transition model:
- $l_1$ and $l_2$, coefficients in Berman (1988)'s $lambda$-heat capacity model
- $k_{\lambda}$, $\frac{{d{T_\lambda }}}{{dP}}$ in Berman (1988)'s $lambda$-transition model
- $T_{\lambda,{P_r}}$, Temperature of the $\lambda$-transition at reference pressure
- $T_{\lambda,{ref}}$, Temperature of the lower bound of the heat capacity integral for the $\lambda$-transition at reference pressure

In [ ]:
l1,l2 = sym.symbols('l1 l2')
kl = sym.symbols('k_lambda')
TlPr, Tlref = sym.symbols('T_lambda_Pr T_lambda_ref')
params = [('l1','(J/m)^(1/2)-K', l1), ('l2', '(J/m)^(1/2)/K^2', l2), ('k_lambda', 'K/bar', kl), 
          ('T_lambda_Pr', 'K', TlPr), ('T_lambda_ref', 'K', Tlref)]

Calculate the transition temperature, $T_{\lambda}$, at the pressure, $P$

In [ ]:
Tl = TlPr + kl*(P-Pr)

Temperature difference between $T_{\lambda}$ at $P$ and $P_r$

In [ ]:
td = TlPr - Tl

Reference temperature for lower limit of heat capacity integral.

In [ ]:
tr = Tlref - td

Heat capacity due to the $\lambda$-transition at $T$ and $P$. Valid: $T_r \le T \le T_{\lambda}$.  

**Note:** The syntax of the arguments to the SymPy Piecewise expression is of the form:
```
(e1,c1), (e2,d2), ..., (eDefault, True)
```
where the sequence is evaluated left to right. *eN* is the value of the resulting expression if the condition *cN* is *True*.  *cN* is always a logical comparison.  The first *cN* that is *True* provides the value of the expression.  If no *cN* evaluate to *True*, then the resulting expression becomes *eDefault* given by the last tuple in the sequence.  See the example below.

In [ ]:
Cpl = (T+td)*(l1+l2*(T+td))**2

Compute the Gibbs free energy of the lambda transition

In [ ]:
Gl = sym.integrate(Cpl,(T,tr,T)) - T*sym.integrate(Cpl/T,(T,tr,T))

... and add this expression to the model

In [ ]:
model.add_expression_to_model(Gl, params, exp_type='restricted', lower_limits=(tr,Pr), upper_limits=(Tl,None))

#### (2) First order phase transition terms
Berman terms valid at T $\ge$ $T_t$. Parameters (in this case $T_t$ is equivalent to $T_{\lambda}$:
- ${{\Delta}_t}H$, First order enthalpy contribution at $T_{\lambda}$

In [ ]:
deltaHt = sym.symbols('H_t')
params = [('H_t','J/m', deltaHt)]

${{\Delta}_t}S = {{\Delta}_t}H/T_{\lambda}$, First order enropy contribution at $T_{\lambda}$: ${{\Delta}_t}H - T {{\Delta}_t}S = -(T-T_{\lambda}) {{\Delta}_t}H/T_{\lambda}$ 

In [ ]:
GaboveTl = -(T-Tl)*deltaHt/Tl

... and add this expression to the model

In [ ]:
model.add_expression_to_model(GaboveTl , params, exp_type='restricted', lower_limits=(Tl,None), upper_limits=(Tl,None))

#### (3) Order-disorder contributions
Parameters of the Berman (1988) order-disorder model:
- $d_0$, $d_1$, $d_2$, $d_3$, $d_4$, $d_5$, order-disorder coefficients from the Berman (1988) model
- $T_{D_{ref}}$, $T_D$, minimum, maximum temperature of ordering interval, $T_{D_{ref}} \le T \le T_D$

In [ ]:
d0,d1,d2,d3,d4,d5 = sym.symbols('d0 d1 d2 d3 d4 d5')
TD,TDref = sym.symbols('T_D T_D_ref')
params = [('d0','J/K-m', d0), ('d1','J/K^(1/2)-m',d1), ('d2','J-K/m',d2), ('d3','J/K^2-m',d3), 
          ('d4','J/K^3-m',d4), ('d5','bar',d5), ('T_D','K',TD), ('T_D_ref','K',TDref)]

In [ ]:
CpDs = d0 + d1/sym.sqrt(T) + d2/T**2 + d3*T + d4*T**2
HDs = sym.integrate(CpDs,(T,TDref,T))
SDs = sym.integrate(CpDs/T,(T,TDref,T))
VDs = HDs/d5
GDs = HDs - T*SDs + VDs*(P-Pr)

In [ ]:
model.add_expression_to_model(GDs , params, exp_type='restricted', lower_limits=(TDref,None), upper_limits=(TD,None))

## Code Print the Model, compile the code and link a Python module
Name the model class

In [ ]:
model.set_module_name('berman')
model.get_berman_std_state_database()

Make a working sub-directory and move down into the directory.  This is done so that generated files will not clash between alternate model configurations.

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

- Choose an existing phase from the Berman database
- Generate an include file and code file for this phase  
  
Note that the call to
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict)
```
generates fast code with unmodifiable model parameters and "calibration-" related functions.  The call to:
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type='calib')
```
generates code suitable for model parameter calibration.  
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"

#### Index 12 in Berman database is High_Albite

In [ ]:
param_dict = model.get_berman_std_state_database(2)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

Save the cython wrappers for use in example notebook #7 (Simple Solution)

In [ ]:
%cp berman.pyx endmembers.pyx

#### Index 56 in Berman database is Anorthite

In [ ]:
param_dict = model.get_berman_std_state_database(6)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

... again, save for notebook #7 (Simple Solution)

In [ ]:
%cat berman.pyx >> endmembers.pyx

#### Diopside from Sack and Ghiorso (1994) quadrilateral pyroxene model

In [ ]:
param_dict = {'Phase': 'DIOPSIDE',
 'Formula': 'MG(1)CA(1)SI(2)O(6)',
 'H_TrPr': -3200583.0,
 'S_TrPr': 142.5,
 'k0': 305.41,
 'k1': -16.049E2,
 'k2': -71.660E5,
 'k3': 92.184E7,
 'V_TrPr': 6.620,
 'v1': -0.872E-6,
 'v2': 1.707E-12,
 'v3': 27.795E-6,
 'v4': 83.082E-10,
 'l1': 0.0,
 'l2': 0.0,
 'k_lambda': 0.0,
 'T_lambda_Pr': 0.0,
 'T_lambda_ref': 0.0,
 'H_t': 0.0,
 'd0': 0.0,
 'd1': 0.0,
 'd2': 0.0,
 'd3': 0.0,
 'd4': 0.0,
 'd5': 0.0,
 'T_D': 0.0,
 'T_D_ref': 0.0,
 'T_r': 298.15,
 'P_r': 1.0}
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

#### Hedenbergite from Sack and Ghiorso (1994) quadrilateral pyroxene model

In [ ]:
param_dict = {'Phase': 'HEDENBERGITE',
 'Formula': 'CA(1)FE(1)SI(2)O(6)',
 'H_TrPr': -2842221.0,
 'S_TrPr': 174.2,
 'k0': 307.89,
 'k1': -15.973e2,
 'k2': -69.925e5,
 'k3': 93.522e7,
 'V_TrPr': 6.7894,
 'v1': -0.9925e-6,
 'v2': 1.4835e-12,
 'v3': 31.371e-6,
 'v4': 83.672e-10,
 'l1': 0.0,
 'l2': 0.0,
 'k_lambda': 0.0,
 'T_lambda_Pr': 0.0,
 'T_lambda_ref': 0.0,
 'H_t': 0.0,
 'd0': 0.0,
 'd1': 0.0,
 'd2': 0.0,
 'd3': 0.0,
 'd4': 0.0,
 'd5': 0.0,
 'T_D': 0.0,
 'T_D_ref': 0.0,
 'T_r': 298.15,
 'P_r': 1.0}
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

#### Enstatite from Sack and Ghiorso (1994) quadrilateral pyroxene model

In [ ]:
param_dict = {'Phase': 'ENSTATITE',
 'Formula': 'MG(2)SI(2)O(6)',
 'H_TrPr': -3086083.0,
 'S_TrPr': 135.164,
 'k0': 333.16,
 'k1': -24.012e2,
 'k2': -45.412e5,
 'k3': 55.830e7,
 'V_TrPr': 6.3279,
 'v1': -0.749e-6,
 'v2': 0.447e-12,
 'v3': 24.656e-6,
 'v4': 74.670e-10,
 'l1': 0.0,
 'l2': 0.0,
 'k_lambda': 0.0,
 'T_lambda_Pr': 0.0,
 'T_lambda_ref': 0.0,
 'H_t': 0.0,
 'd0': 0.0,
 'd1': 0.0,
 'd2': 0.0,
 'd3': 0.0,
 'd4': 0.0,
 'd5': 0.0,
 'T_D': 0.0,
 'T_D_ref': 0.0,
 'T_r': 298.15,
 'P_r': 1.0}
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

#### Index 42 in Berman database is Potassium_Feldspar

In [ ]:
param_dict = model.get_berman_std_state_database(47)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

... final endmember for example notebook #7 (Simple Solution)

In [ ]:
%cat berman.pyx >> endmembers.pyx

## Import the new module and test the model

In [ ]:
import berman
%cd ..

Evaluate functions at temperature (K) and pressure (bars)

In [ ]:
t = 1000.0
p = 10000.0

### Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(berman.cy_Potassium_Feldspar_berman_identifier())
    print(berman.cy_Potassium_Feldspar_berman_name())
    print(berman.cy_Potassium_Feldspar_berman_formula())
    print(berman.cy_Potassium_Feldspar_berman_mw())
    print(berman.cy_Potassium_Feldspar_berman_elements())
except AttributeError:
    pass
try:
    print(berman.cy_Potassium_Feldspar_berman_calib_identifier())
    print(berman.cy_Potassium_Feldspar_berman_calib_name())
    print(berman.cy_Potassium_Feldspar_berman_calib_formula())
    print(berman.cy_Potassium_Feldspar_berman_calib_mw())
    print(berman.cy_Potassium_Feldspar_berman_calib_elements())
except AttributeError:
    pass

Execute the standard thermodynamic property retrieval functions:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
try:
    print(fmt.format('G', berman.cy_Potassium_Feldspar_berman_g(t,p), 'J/m'))
    print(fmt.format('dGdT', berman.cy_Potassium_Feldspar_berman_dgdt(t,p), 'J/K-m'))
    print(fmt.format('dGdP', berman.cy_Potassium_Feldspar_berman_dgdp(t,p), 'J/bar-m'))
    print(fmt.format('d2GdP2', berman.cy_Potassium_Feldspar_berman_d2gdt2(t,p), 'J/K^2-m'))
    print(fmt.format('d2GdTdP', berman.cy_Potassium_Feldspar_berman_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', berman.cy_Potassium_Feldspar_berman_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', berman.cy_Potassium_Feldspar_berman_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', berman.cy_Potassium_Feldspar_berman_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', berman.cy_Potassium_Feldspar_berman_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', berman.cy_Potassium_Feldspar_berman_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmt.format('S', berman.cy_Potassium_Feldspar_berman_s(t,p), 'J/K-m'))
    print(fmt.format('V', berman.cy_Potassium_Feldspar_berman_v(t,p), 'J/bar-m'))
    print(fmt.format('Cv', berman.cy_Potassium_Feldspar_berman_cv(t,p), 'J/K-m'))
    print(fmt.format('Cp', berman.cy_Potassium_Feldspar_berman_cp(t,p), 'J/K-m'))
    print(fmt.format('dCpdT', berman.cy_Potassium_Feldspar_berman_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', berman.cy_Potassium_Feldspar_berman_alpha(t,p), '1/K'))
    print(fmt.format('beta', berman.cy_Potassium_Feldspar_berman_beta(t,p), '1/bar'))
    print(fmt.format('K', berman.cy_Potassium_Feldspar_berman_K(t,p), 'bar'))
    print(fmt.format('Kp', berman.cy_Potassium_Feldspar_berman_Kp(t,p), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', berman.cy_Potassium_Feldspar_berman_calib_g(t,p), 'J/m'))
    print(fmt.format('dGdT', berman.cy_Potassium_Feldspar_berman_calib_dgdt(t,p), 'J/K-m'))
    print(fmt.format('dGdP', berman.cy_Potassium_Feldspar_berman_calib_dgdp(t,p), 'J/bar-m'))
    print(fmt.format('d2GdP2', berman.cy_Potassium_Feldspar_berman_calib_d2gdt2(t,p), 'J/K^2-m'))
    print(fmt.format('d2GdTdP', berman.cy_Potassium_Feldspar_berman_calib_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', berman.cy_Potassium_Feldspar_berman_calib_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', berman.cy_Potassium_Feldspar_berman_calib_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', berman.cy_Potassium_Feldspar_berman_calib_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', berman.cy_Potassium_Feldspar_berman_calib_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', berman.cy_Potassium_Feldspar_berman_calib_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmt.format('S', berman.cy_Potassium_Feldspar_berman_calib_s(t,p), 'J/K-m'))
    print(fmt.format('V', berman.cy_Potassium_Feldspar_berman_calib_v(t,p), 'J/bar-m'))
    print(fmt.format('Cv', berman.cy_Potassium_Feldspar_berman_calib_cv(t,p), 'J/K-m'))
    print(fmt.format('Cp', berman.cy_Potassium_Feldspar_berman_calib_cp(t,p), 'J/K-m'))
    print(fmt.format('dCpdT', berman.cy_Potassium_Feldspar_berman_calib_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', berman.cy_Potassium_Feldspar_berman_calib_alpha(t,p), '1/K'))
    print(fmt.format('beta', berman.cy_Potassium_Feldspar_berman_calib_beta(t,p), '1/bar'))
    print(fmt.format('K', berman.cy_Potassium_Feldspar_berman_calib_K(t,p), 'bar'))
    print(fmt.format('Kp', berman.cy_Potassium_Feldspar_berman_calib_Kp(t,p), ''))
except AttributeError:
    pass

### Available only in the "Calib" versions of generated code
Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
try:
    np = berman.cy_Potassium_Feldspar_berman_get_param_number()
    names = berman.cy_Potassium_Feldspar_berman_get_param_names()
    units = berman.cy_Potassium_Feldspar_berman_get_param_units()
    values = berman.cy_Potassium_Feldspar_berman_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], berman.cy_Potassium_Feldspar_berman_get_param_value(i), units[i]))
except AttributeError:
    pass

Test the functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    berman.cy_Potassium_Feldspar_berman_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], berman.cy_Potassium_Feldspar_berman_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

Test the functions that allow modification of a particular parameter value

In [ ]:
try:
    berman.cy_Potassium_Feldspar_berman_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], berman.cy_Potassium_Feldspar_berman_get_param_value(i), units[i]))
except AttributeError:
    pass

Evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', berman.cy_Potassium_Feldspar_berman_dparam_g(t, p, i)))
        print (fmt.format('dGdT', berman.cy_Potassium_Feldspar_berman_dparam_dgdt(t, p, i)))
        print (fmt.format('dGdP', berman.cy_Potassium_Feldspar_berman_dparam_dgdp(t, p, i)))
        print (fmt.format('d2GdT2', berman.cy_Potassium_Feldspar_berman_dparam_d2gdt2(t, p, i)))
        print (fmt.format('d2GdTdP', berman.cy_Potassium_Feldspar_berman_dparam_d2gdtdp(t, p, i)))
        print (fmt.format('d2GdP2', berman.cy_Potassium_Feldspar_berman_dparam_d2gdp2(t, p, i)))
        print (fmt.format('d3GdT3', berman.cy_Potassium_Feldspar_berman_dparam_d3gdt3(t, p, i)))
        print (fmt.format('d3GdT2dP', berman.cy_Potassium_Feldspar_berman_dparam_d3gdt2dp(t, p, i)))
        print (fmt.format('d3GdTdP2', berman.cy_Potassium_Feldspar_berman_dparam_d3gdtdp2(t, p, i)))
        print (fmt.format('d3GdP3', berman.cy_Potassium_Feldspar_berman_dparam_d3gdp3(t, p, i)))
except (AttributeError, TypeError):
    pass

## Test the generated code against the standard Berman code base

In [ ]:
from thermoengine import model

In [ ]:
bermanDB = model.Database()

In [ ]:
abbrv = ""
for full_name, abbrv in zip(bermanDB.phase_info.phase_name,bermanDB.phase_info.abbrev):
    if phase_name == full_name:
        break
refModel = bermanDB.get_phase(abbrv)

In [ ]:
import math
fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:13.6e} {4:6.2f}%"
fmts = "{0:<10.10s} {1:13.6e}"
try:
    x = berman.cy_Potassium_Feldspar_berman_g(t,p)
    y = refModel.gibbs_energy(t,p)
    print(fmt.format('G', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_dgdt(t,p)
    y = -refModel.entropy(t,p)
    print(fmt.format('dGdT', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_dgdp(t,p)
    y = refModel.volume(t,p)
    print(fmt.format('dGdP', x, y, x-y, 100.0*math.fabs((x-y)/y))) 
    x = berman.cy_Potassium_Feldspar_berman_d2gdt2(t,p)
    print(fmts.format('d2GdT2', x))
    x = berman.cy_Potassium_Feldspar_berman_d2gdtdp(t,p)
    print(fmts.format('d2GdTdP', x))
    x = berman.cy_Potassium_Feldspar_berman_d2gdp2(t,p)
    print(fmts.format('d2GdP2', x))
    x = berman.cy_Potassium_Feldspar_berman_d3gdt3(t,p)
    print(fmts.format('d3GdT3', x))
    x = berman.cy_Potassium_Feldspar_berman_d3gdt2dp(t,p)
    print(fmts.format('d3GdT2dP', x))
    x = berman.cy_Potassium_Feldspar_berman_d3gdtdp2(t,p)
    print(fmts.format('d3GdTdP2', x))
    x = berman.cy_Potassium_Feldspar_berman_d3gdp3(t,p)
    print(fmts.format('d3GdP3', x))
    x = berman.cy_Potassium_Feldspar_berman_s(t,p)
    y = refModel.entropy(t,p)
    print(fmt.format('S', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_v(t,p)
    y = refModel.volume(t,p)
    print(fmt.format('V', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_cv(t,p)
    print(fmts.format('Cv', x))
    x = berman.cy_Potassium_Feldspar_berman_cp(t,p)
    y = refModel.heat_capacity(t,p)
    print(fmt.format('Cp', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_dcpdt(t,p)
    print(fmts.format('dCpdT', x))
    x = berman.cy_Potassium_Feldspar_berman_alpha(t,p)
    print(fmts.format('alpha', x))
    x = berman.cy_Potassium_Feldspar_berman_beta(t,p)
    print(fmts.format('beta', x))
    x = berman.cy_Potassium_Feldspar_berman_K(t,p)
    print(fmts.format('K', x))
    x = berman.cy_Potassium_Feldspar_berman_Kp(t,p)
    print(fmts.format('Kp', x))
except AttributeError:
    pass
try:
    x = berman.cy_Potassium_Feldspar_berman_calib_g(t,p)
    y = refModel.gibbs_energy(t,p)
    print(fmt.format('G', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_calib_dgdt(t,p)
    y = -refModel.entropy(t,p)
    print(fmt.format('dGdT', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_calib_dgdp(t,p)
    y = refModel.volume(t,p)
    print(fmt.format('dGdP', x, y, x-y, 100.0*math.fabs((x-y)/y))) 
    x = berman.cy_Potassium_Feldspar_berman_calib_d2gdt2(t,p)
    print(fmts.format('d2GdT2', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d2gdtdp(t,p)
    print(fmts.format('d2GdTdP', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d2gdp2(t,p)
    print(fmts.format('d2GdP2', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d3gdt3(t,p)
    print(fmts.format('d3GdT3', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d3gdt2dp(t,p)
    print(fmts.format('d3GdT2dP', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d3gdtdp2(t,p)
    print(fmts.format('d3GdTdP2', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_d3gdp3(t,p)
    print(fmts.format('d3GdP3', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_s(t,p)
    y = refModel.entropy(t,p)
    print(fmt.format('S', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_calib_v(t,p)
    y = refModel.volume(t,p)
    print(fmt.format('V', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_calib_cv(t,p)
    print(fmts.format('Cv', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_cp(t,p)
    y = refModel.heat_capacity(t,p)
    print(fmt.format('Cp', x, y, x-y, 100.0*math.fabs((x-y)/y)))
    x = berman.cy_Potassium_Feldspar_berman_calib_dcpdt(t,p)
    print(fmts.format('dCpdT', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_alpha(t,p)
    print(fmts.format('alpha', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_beta(t,p)
    print(fmts.format('beta', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_K(t,p)
    print(fmts.format('K', x))
    x = berman.cy_Potassium_Feldspar_berman_calib_Kp(t,p)
    print(fmts.format('Kp', x))
except AttributeError:
    pass

## Time execution of the code

In [ ]:
try:
    %timeit berman.cy_Potassium_Feldspar_berman_calib_g(t,p)
except AttributeError:
    pass
try:
    %timeit berman.cy_Potassium_Feldspar_berman_g(t,p)
except AttributeError:
    pass

In [ ]:
%timeit refModel.gibbs_energy(t,p)